<a href="https://colab.research.google.com/github/gershomrichardbruno/Ergonomics-Smart-Chair/blob/main/Body_posture_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Ergonomics Chair Analysis System - Google Colab Implementation
# Complete system for analyzing seating positions, angles, and acceleration from video markers

# ================================================================================================
# SECTION 1: INSTALLATION & IMPORTS
# ================================================================================================

# Install required packages
!pip install opencv-python mediapipe numpy matplotlib pandas plotly scipy scikit-learn
!pip install ultralytics  # For YOLO if needed for marker detection
!apt-get update && apt-get install -y libgl1-mesa-glx  # For OpenCV

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import mediapipe as mp
from scipy import signal
from scipy.interpolate import interp1d
from sklearn.cluster import KMeans
import json
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ All packages installed and imported successfully!")

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [8]:
upload_and_analyze()

📁 Please upload your video file(s):


Saving 4937030-uhd_2160_3840_24fps.mp4 to 4937030-uhd_2160_3840_24fps.mp4

🎬 Analyzing 4937030-uhd_2160_3840_24fps.mp4...
🚀 Starting Ergonomics Analysis...
📹 Video: 4937030-uhd_2160_3840_24fps.mp4
📹 Processing video: 24.0 FPS, 203 frames
Progress: 14.8% - Frame 30/203
Progress: 29.6% - Frame 60/203
Progress: 44.3% - Frame 90/203
Progress: 59.1% - Frame 120/203
Progress: 73.9% - Frame 150/203
Progress: 88.7% - Frame 180/203
✅ Video processing completed!

📊 ANALYSIS SUMMARY:
Total Frames: 203
Duration Seconds: 8.46
Avg Seat Angle: 81.18
Max Seat Angle: 90.82
Min Seat Angle: 75.39
Angle Std: 3.02
Max Acceleration: 3070.45
Avg Acceleration: 366.76
Marker Detection Rate: 100.00

📈 Generating visualizations...


✅ Results exported to ergonomics_analysis_4937030-uhd_2160_3840_24fps_20250806_201940.json and ergonomics_analysis_4937030-uhd_2160_3840_24fps_20250806_201940.csv
✅ Analysis complete for 4937030-uhd_2160_3840_24fps.mp4


In [9]:
# ================================================================================================
# SECTION 2: MARKER DETECTION CLASS
# ================================================================================================

class MarkerDetector:
    """
    Advanced marker detection system for ergonomics analysis
    """

    def __init__(self):
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(
            static_image_mode=False,
            model_complexity=2,
            enable_segmentation=False,
            min_detection_confidence=0.5
        )
        self.mp_drawing = mp.solutions.drawing_utils

        # Color-based marker detection parameters
        self.marker_colors = {
            'red': {'lower': np.array([0, 100, 100]), 'upper': np.array([10, 255, 255])},
            'green': {'lower': np.array([50, 100, 100]), 'upper': np.array([70, 255, 255])},
            'blue': {'lower': np.array([100, 100, 100]), 'upper': np.array([130, 255, 255])},
            'yellow': {'lower': np.array([20, 100, 100]), 'upper': np.array([30, 255, 255])}
        }

        self.detected_markers = {}
        self.marker_history = []

    def detect_colored_markers(self, frame):
        """
        Detect colored markers in the frame
        """
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        markers = {}

        for color_name, color_range in self.marker_colors.items():
            # Create mask for the color
            mask = cv2.inRange(hsv, color_range['lower'], color_range['upper'])

            # Find contours
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if contours:
                # Find the largest contour (assuming it's our marker)
                largest_contour = max(contours, key=cv2.contourArea)

                if cv2.contourArea(largest_contour) > 100:  # Minimum size threshold
                    # Get center of the marker
                    M = cv2.moments(largest_contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        area = cv2.contourArea(largest_contour)

                        markers[color_name] = {
                            'x': cx,
                            'y': cy,
                            'area': area,
                            'confidence': min(area / 1000, 1.0)  # Normalized confidence
                        }

        return markers

    def detect_pose_landmarks(self, frame):
        """
        Detect human pose landmarks using MediaPipe
        """
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.pose.process(rgb_frame)

        landmarks = {}
        if results.pose_landmarks:
            for idx, landmark in enumerate(results.pose_landmarks.landmark):
                if landmark.visibility > 0.5:  # Only consider visible landmarks
                    landmarks[f'pose_{idx}'] = {
                        'x': int(landmark.x * frame.shape[1]),
                        'y': int(landmark.y * frame.shape[0]),
                        'z': landmark.z,
                        'confidence': landmark.visibility
                    }

        return landmarks, results

    def detect_all_markers(self, frame):
        """
        Detect both colored markers and pose landmarks
        """
        colored_markers = self.detect_colored_markers(frame)
        pose_landmarks, pose_results = self.detect_pose_landmarks(frame)

        # Combine all markers
        all_markers = {**colored_markers, **pose_landmarks}

        return all_markers, pose_results

In [10]:

# ================================================================================================
# SECTION 3: ANGLE CALCULATION CLASS
# ================================================================================================

class AngleCalculator:
    """
    Calculate seat angles and body posture angles from detected markers
    """

    def __init__(self):
        self.angle_history = []
        self.velocity_history = []
        self.acceleration_history = []

    def calculate_angle_between_points(self, p1, p2, p3):
        """
        Calculate angle between three points (p2 is the vertex)
        """
        # Convert to numpy arrays
        p1 = np.array([p1['x'], p1['y']])
        p2 = np.array([p2['x'], p2['y']])
        p3 = np.array([p3['x'], p3['y']])

        # Calculate vectors
        v1 = p1 - p2
        v2 = p3 - p2

        # Calculate angle
        cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        cos_angle = np.clip(cos_angle, -1, 1)  # Ensure valid range for arccos
        angle = np.arccos(cos_angle)

        return np.degrees(angle)

    def calculate_seat_angle(self, markers):
        """
        Calculate the seat backrest angle
        """
        # Try to use colored markers first (more reliable for seat angle)
        if 'red' in markers and 'blue' in markers:
            # Assuming red marker is on seat base, blue on backrest
            seat_base = markers['red']
            backrest = markers['blue']

            # Calculate angle relative to horizontal
            dx = backrest['x'] - seat_base['x']
            dy = backrest['y'] - seat_base['y']
            angle = np.degrees(np.arctan2(dy, dx))

            # Normalize to typical seat angle range (0-180 degrees)
            if angle < 0:
                angle += 180

            return angle

        # Fallback to pose landmarks if colored markers not available
        elif 'pose_11' in markers and 'pose_23' in markers:  # Hip and shoulder
            return self.calculate_angle_between_points(
                markers['pose_23'],  # Hip
                markers['pose_11'],  # Shoulder
                {'x': markers['pose_11']['x'] + 100, 'y': markers['pose_11']['y']}  # Reference point
            )

        return None

    def calculate_body_angles(self, markers):
        """
        Calculate various body posture angles
        """
        angles = {}

        # Spine angle (if pose landmarks available)
        if 'pose_11' in markers and 'pose_23' in markers:  # Shoulder to hip
            spine_angle = self.calculate_angle_between_points(
                {'x': markers['pose_11']['x'], 'y': 0},  # Vertical reference
                markers['pose_11'],  # Shoulder
                markers['pose_23']   # Hip
            )
            angles['spine_angle'] = 180 - spine_angle  # Convert to more intuitive measurement

        # Neck angle
        if 'pose_0' in markers and 'pose_11' in markers:  # Nose to shoulder
            neck_angle = self.calculate_angle_between_points(
                {'x': markers['pose_11']['x'], 'y': 0},  # Vertical reference
                markers['pose_11'],  # Shoulder
                markers['pose_0']    # Nose
            )
            angles['neck_angle'] = neck_angle

        return angles

    def calculate_motion_metrics(self, current_angle, timestamp):
        """
        Calculate velocity and acceleration from angle changes
        """
        self.angle_history.append({'angle': current_angle, 'time': timestamp})

        if len(self.angle_history) < 2:
            return 0, 0

        # Calculate velocity (angular velocity)
        dt = self.angle_history[-1]['time'] - self.angle_history[-2]['time']
        if dt > 0:
            da = self.angle_history[-1]['angle'] - self.angle_history[-2]['angle']
            velocity = da / dt
            self.velocity_history.append({'velocity': velocity, 'time': timestamp})
        else:
            velocity = 0

        # Calculate acceleration
        if len(self.velocity_history) < 2:
            acceleration = 0
        else:
            dt_v = self.velocity_history[-1]['time'] - self.velocity_history[-2]['time']
            if dt_v > 0:
                dv = self.velocity_history[-1]['velocity'] - self.velocity_history[-2]['velocity']
                acceleration = dv / dt_v
            else:
                acceleration = 0

        self.acceleration_history.append({'acceleration': acceleration, 'time': timestamp})

        return velocity, acceleration

In [11]:
# ================================================================================================
# SECTION 4: VIDEO PROCESSOR CLASS
# ================================================================================================

class VideoProcessor:
    """
    Main video processing class for ergonomics analysis
    """

    def __init__(self):
        self.marker_detector = MarkerDetector()
        self.angle_calculator = AngleCalculator()
        self.results = {
            'frames': [],
            'angles': [],
            'markers': [],
            'accelerations': [],
            'timestamps': []
        }

    def process_video(self, video_path, output_path=None, subject_info=None):
        """
        Process video and extract ergonomics data
        """
        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            raise ValueError("Error opening video file")

        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        print(f"📹 Processing video: {fps} FPS, {total_frames} frames")

        # Prepare output video if path provided
        if output_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, fps,
                                (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                                 int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

        frame_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1
            timestamp = frame_count / fps

            # Detect markers
            markers, pose_results = self.marker_detector.detect_all_markers(frame)

            # Calculate angles
            seat_angle = self.angle_calculator.calculate_seat_angle(markers)
            body_angles = self.angle_calculator.calculate_body_angles(markers)

            # Calculate motion metrics
            if seat_angle:
                velocity, acceleration = self.angle_calculator.calculate_motion_metrics(seat_angle, timestamp)
            else:
                velocity, acceleration = 0, 0

            # Draw markers and annotations on frame
            annotated_frame = self.draw_annotations(frame, markers, seat_angle, acceleration, pose_results)

            # Store results
            self.results['frames'].append(frame_count)
            self.results['angles'].append(seat_angle if seat_angle else 0)
            self.results['markers'].append(len(markers))
            self.results['accelerations'].append(acceleration)
            self.results['timestamps'].append(timestamp)

            # Write annotated frame
            if output_path:
                out.write(annotated_frame)

            # Progress indicator
            if frame_count % 30 == 0:  # Every second at 30fps
                progress = (frame_count / total_frames) * 100
                print(f"Progress: {progress:.1f}% - Frame {frame_count}/{total_frames}")

        cap.release()
        if output_path:
            out.release()

        print("✅ Video processing completed!")
        return self.results

    def draw_annotations(self, frame, markers, seat_angle, acceleration, pose_results=None):
        """
        Draw markers, angles, and other annotations on frame
        """
        annotated_frame = frame.copy()

        # Draw colored markers
        for color, marker in markers.items():
            if color in ['red', 'green', 'blue', 'yellow']:
                # Draw marker circle
                cv2.circle(annotated_frame, (marker['x'], marker['y']),
                          15, (0, 255, 0), 3)

                # Draw marker label
                cv2.putText(annotated_frame, f"{color.upper()}",
                           (marker['x'] - 30, marker['y'] - 25),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Draw pose landmarks
        if pose_results and pose_results.pose_landmarks:
            self.marker_detector.mp_drawing.draw_landmarks(
                annotated_frame, pose_results.pose_landmarks,
                self.marker_detector.mp_pose.POSE_CONNECTIONS)

        # Draw angle information
        if seat_angle:
            cv2.putText(annotated_frame, f"Seat Angle: {seat_angle:.1f}°",
                       (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        cv2.putText(annotated_frame, f"Acceleration: {acceleration:.2f} deg/s²",
                   (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        cv2.putText(annotated_frame, f"Markers: {len(markers)}",
                   (20, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        return annotated_frame

In [12]:
# ================================================================================================
# SECTION 5: DATA ANALYSIS & VISUALIZATION
# ================================================================================================

class DataAnalyzer:
    """
    Analyze and visualize ergonomics data
    """

    def __init__(self, results_data):
        self.data = results_data
        self.df = pd.DataFrame(results_data)

    def generate_summary_statistics(self):
        """
        Generate comprehensive summary statistics
        """
        stats = {
            'total_frames': len(self.df),
            'duration_seconds': self.df['timestamps'].max(),
            'avg_seat_angle': self.df['angles'].mean(),
            'max_seat_angle': self.df['angles'].max(),
            'min_seat_angle': self.df['angles'].min(),
            'angle_std': self.df['angles'].std(),
            'max_acceleration': self.df['accelerations'].max(),
            'avg_acceleration': np.abs(self.df['accelerations']).mean(),
            'marker_detection_rate': (self.df['markers'] > 0).mean() * 100
        }
        return stats

    def plot_comprehensive_analysis(self):
        """
        Create comprehensive analysis plots
        """
        fig = make_subplots(
            rows=3, cols=2,
            subplot_titles=('Seat Angle Over Time', 'Acceleration Over Time',
                          'Angle Distribution', 'Acceleration Distribution',
                          'Marker Detection Rate', 'Angle vs Acceleration'),
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"secondary_y": False}]]
        )

        # 1. Seat Angle Over Time
        fig.add_trace(
            go.Scatter(x=self.df['timestamps'], y=self.df['angles'],
                      mode='lines', name='Seat Angle',
                      line=dict(color='blue', width=2)),
            row=1, col=1
        )

        # 2. Acceleration Over Time
        fig.add_trace(
            go.Scatter(x=self.df['timestamps'], y=self.df['accelerations'],
                      mode='lines', name='Acceleration',
                      line=dict(color='red', width=2)),
            row=1, col=2
        )

        # 3. Angle Distribution
        fig.add_trace(
            go.Histogram(x=self.df['angles'], name='Angle Distribution',
                        marker_color='blue', opacity=0.7),
            row=2, col=1
        )

        # 4. Acceleration Distribution
        fig.add_trace(
            go.Histogram(x=self.df['accelerations'], name='Acceleration Distribution',
                        marker_color='red', opacity=0.7),
            row=2, col=2
        )

        # 5. Marker Detection Rate
        marker_rate = self.df.groupby(pd.cut(self.df['timestamps'], bins=20))['markers'].mean()
        fig.add_trace(
            go.Bar(x=[str(i) for i in marker_rate.index], y=marker_rate.values,
                  name='Avg Markers Detected', marker_color='green'),
            row=3, col=1
        )

        # 6. Angle vs Acceleration Scatter
        fig.add_trace(
            go.Scatter(x=self.df['angles'], y=self.df['accelerations'],
                      mode='markers', name='Angle vs Acceleration',
                      marker=dict(color='purple', size=4, opacity=0.6)),
            row=3, col=2
        )

        # Update layout
        fig.update_layout(
            height=1000,
            title_text="Comprehensive Ergonomics Analysis",
            showlegend=False
        )

        # Update axes labels
        fig.update_xaxes(title_text="Time (seconds)", row=1, col=1)
        fig.update_xaxes(title_text="Time (seconds)", row=1, col=2)
        fig.update_xaxes(title_text="Angle (degrees)", row=2, col=1)
        fig.update_xaxes(title_text="Acceleration (deg/s²)", row=2, col=2)
        fig.update_xaxes(title_text="Time Bins", row=3, col=1)
        fig.update_xaxes(title_text="Angle (degrees)", row=3, col=2)

        fig.update_yaxes(title_text="Angle (degrees)", row=1, col=1)
        fig.update_yaxes(title_text="Acceleration (deg/s²)", row=1, col=2)
        fig.update_yaxes(title_text="Frequency", row=2, col=1)
        fig.update_yaxes(title_text="Frequency", row=2, col=2)
        fig.update_yaxes(title_text="Avg Markers", row=3, col=1)
        fig.update_yaxes(title_text="Acceleration (deg/s²)", row=3, col=2)

        return fig

    def export_results(self, filename, subject_info=None):
        """
        Export analysis results to JSON and CSV
        """
        # Prepare export data
        export_data = {
            'subject_info': subject_info or {},
            'summary_statistics': self.generate_summary_statistics(),
            'raw_data': self.data,
            'analysis_timestamp': datetime.now().isoformat()
        }

        # Export JSON
        with open(f"{filename}.json", 'w') as f:
            json.dump(export_data, f, indent=2, default=str)

        # Export CSV
        self.df.to_csv(f"{filename}.csv", index=False)

        print(f"✅ Results exported to {filename}.json and {filename}.csv")


In [13]:
# ================================================================================================
# SECTION 6: MAIN USAGE EXAMPLE
# ================================================================================================

def analyze_ergonomics_video(video_path, subject_info=None, export_results=True):
    """
    Main function to analyze ergonomics video

    Args:
        video_path: Path to the input video
        subject_info: Dictionary with subject information
        export_results: Whether to export results
    """

    print("🚀 Starting Ergonomics Analysis...")
    print(f"📹 Video: {video_path}")

    # Initialize processor
    processor = VideoProcessor()

    # Process video
    results = processor.process_video(
        video_path,
        output_path=f"analyzed_{os.path.basename(video_path)}",
        subject_info=subject_info
    )

    # Analyze data
    analyzer = DataAnalyzer(results)

    # Generate summary
    stats = analyzer.generate_summary_statistics()
    print("\n📊 ANALYSIS SUMMARY:")
    print("=" * 50)
    for key, value in stats.items():
        if isinstance(value, float):
            print(f"{key.replace('_', ' ').title()}: {value:.2f}")
        else:
            print(f"{key.replace('_', ' ').title()}: {value}")

    # Create visualizations
    print("\n📈 Generating visualizations...")
    fig = analyzer.plot_comprehensive_analysis()
    fig.show()

    # Export results if requested
    if export_results:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        subject_id = subject_info.get('id', 'unknown') if subject_info else 'unknown'
        filename = f"ergonomics_analysis_{subject_id}_{timestamp}"
        analyzer.export_results(filename, subject_info)

    return results, analyzer

# ================================================================================================
# EXAMPLE USAGE
# ================================================================================================

# Example subject information
sample_subject_info = {
    'id': 'Subject_001',
    'age': 28,
    'height_cm': 175,
    'weight_kg': 70,
    'session_date': '2025-01-15',
    'notes': 'Normal seating session with angle changes'
}

print("""
🪑 ERGONOMICS CHAIR ANALYSIS SYSTEM READY!

To use this system:

1. Upload your video file to Colab:
   from google.colab import files
   uploaded = files.upload()
   video_path = list(uploaded.keys())[0]

2. Run the analysis:
   results, analyzer = analyze_ergonomics_video(
       video_path=video_path,
       subject_info=sample_subject_info,
       export_results=True
   )

3. For multiple subjects:
   subjects = ['video1.mp4', 'video2.mp4', 'video3.mp4']
   all_results = []

   for i, video in enumerate(subjects):
       subject_info = sample_subject_info.copy()
       subject_info['id'] = f'Subject_{i+1:03d}'

       results, analyzer = analyze_ergonomics_video(video, subject_info)
       all_results.append(results)

The system will:
✅ Detect colored markers and pose landmarks
✅ Calculate seat angles and body posture
✅ Compute acceleration during angle changes
✅ Generate comprehensive visualizations
✅ Export data in JSON and CSV formats
✅ Create annotated video with overlay information

Ready to process your videos! 🎬
""")

# ================================================================================================
# ADVANCED FEATURES
# ================================================================================================

class BatchProcessor:
    """
    Process multiple videos and subjects
    """

    def __init__(self):
        self.all_results = []
        self.summary_df = None

    def process_multiple_subjects(self, video_list, subject_list=None):
        """
        Process multiple videos with subject information
        """
        results = []

        for i, video_path in enumerate(video_list):
            print(f"\n🔄 Processing Subject {i+1}/{len(video_list)}")

            subject_info = subject_list[i] if subject_list else {
                'id': f'Subject_{i+1:03d}',
                'session': i+1
            }

            try:
                result, analyzer = analyze_ergonomics_video(
                    video_path, subject_info, export_results=False
                )

                # Add subject info to results
                result['subject_info'] = subject_info
                results.append(result)

            except Exception as e:
                print(f"❌ Error processing {video_path}: {str(e)}")
                continue

        self.all_results = results
        self.create_batch_summary()

        return results

    def create_batch_summary(self):
        """
        Create summary across all subjects
        """
        summary_data = []

        for result in self.all_results:
            subject_info = result.get('subject_info', {})

            # Calculate statistics for this subject
            df = pd.DataFrame(result)
            stats = {
                'subject_id': subject_info.get('id', 'unknown'),
                'age': subject_info.get('age', 0),
                'height': subject_info.get('height_cm', 0),
                'weight': subject_info.get('weight_kg', 0),
                'duration': df['timestamps'].max(),
                'avg_angle': df['angles'].mean(),
                'angle_range': df['angles'].max() - df['angles'].min(),
                'max_acceleration': df['accelerations'].max(),
                'avg_abs_acceleration': np.abs(df['accelerations']).mean(),
                'total_angle_changes': len(df[df['accelerations'].abs() > 1])
            }

            summary_data.append(stats)

        self.summary_df = pd.DataFrame(summary_data)

        return self.summary_df

    def plot_comparative_analysis(self):
        """
        Create comparative plots across subjects
        """
        if self.summary_df is None:
            print("❌ No summary data available. Run process_multiple_subjects first.")
            return

        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Average Angle by Subject', 'Max Acceleration by Subject',
                          'Age vs Average Angle', 'Angle Range Distribution')
        )

        # 1. Average angle by subject
        fig.add_trace(
            go.Bar(x=self.summary_df['subject_id'], y=self.summary_df['avg_angle'],
                  name='Avg Angle', marker_color='blue'),
            row=1, col=1
        )

        # 2. Max acceleration by subject
        fig.add_trace(
            go.Bar(x=self.summary_df['subject_id'], y=self.summary_df['max_acceleration'],
                  name='Max Acceleration', marker_color='red'),
            row=1, col=2
        )

        # 3. Age vs Average Angle
        fig.add_trace(
            go.Scatter(x=self.summary_df['age'], y=self.summary_df['avg_angle'],
                      mode='markers', name='Age vs Angle',
                      marker=dict(size=10, color='green')),
            row=2, col=1
        )

        # 4. Angle Range Distribution
        fig.add_trace(
            go.Histogram(x=self.summary_df['angle_range'], name='Angle Range',
                        marker_color='purple', opacity=0.7),
            row=2, col=2
        )

        fig.update_layout(
            height=800,
            title_text="Comparative Analysis Across Subjects",
            showlegend=False
        )

        return fig

print("\n🔧 Advanced batch processing tools loaded!")
print("Use BatchProcessor() for analyzing multiple subjects simultaneously.")

# Helper function for easy video upload
def upload_and_analyze():
    """
    Helper function for easy video upload and analysis in Colab
    """
    print("📁 Please upload your video file(s):")
    from google.colab import files
    uploaded = files.upload()

    for filename in uploaded.keys():
        print(f"\n🎬 Analyzing {filename}...")

        # Create subject info
        subject_info = {
            'id': filename.split('.')[0],
            'upload_timestamp': datetime.now().isoformat(),
            'filename': filename
        }

        try:
            results, analyzer = analyze_ergonomics_video(
                filename, subject_info, export_results=True
            )
            print(f"✅ Analysis complete for {filename}")

        except Exception as e:
            print(f"❌ Error analyzing {filename}: {str(e)}")

print("\n🚀 Ready to start! Use upload_and_analyze() to begin processing your videos.")


🪑 ERGONOMICS CHAIR ANALYSIS SYSTEM READY!

To use this system:

1. Upload your video file to Colab:
   from google.colab import files
   uploaded = files.upload()
   video_path = list(uploaded.keys())[0]

2. Run the analysis:
   results, analyzer = analyze_ergonomics_video(
       video_path=video_path,
       subject_info=sample_subject_info,
       export_results=True
   )

3. For multiple subjects:
   subjects = ['video1.mp4', 'video2.mp4', 'video3.mp4']
   all_results = []
   
   for i, video in enumerate(subjects):
       subject_info = sample_subject_info.copy()
       subject_info['id'] = f'Subject_{i+1:03d}'
       
       results, analyzer = analyze_ergonomics_video(video, subject_info)
       all_results.append(results)

The system will:
✅ Detect colored markers and pose landmarks
✅ Calculate seat angles and body posture
✅ Compute acceleration during angle changes
✅ Generate comprehensive visualizations
✅ Export data in JSON and CSV formats
✅ Create annotated video with over